#  Setup stuff: Connectivity

In [1]:

from tabulate import tabulate
import pandas as pd
   #
pd.set_option("display.width", 480)

print("--")


--


In [2]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [4]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [5]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, JAexCvLa3XMgCTEZbPfNJfo8Xq7SZsscbQh9wA4bmvcE, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [6]:
import pandas as pd

print("--")


--


In [7]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport", 600],
   ["ORD", "Chicago O-Hare"          , "Airport", 601],
   ["SJC", "San Jose"                , "Airport", 602],
   ["DEN", "Denver"                  , "Airport", 603],
       #
   ], columns = ["airport_code", "airport_name", "LABEL", "internal_id"])

df_airports.head(20)


,airport_code,airport_name,LABEL,internal_id
0,MKE,Milwaukee,Airport,600
1,ORD,Chicago O-Hare,Airport,601
2,SJC,San Jose,Airport,602
3,DEN,Denver,Airport,603


In [8]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

df_flights.head(30)


,START_ID,END_ID,DISTANCE,NUM_HOPS,TYPE
0,MKE,ORD,66,1,FLIES_TO
1,ORD,MKE,66,1,FLIES_TO
2,ORD,DEN,886,1,FLIES_TO
3,DEN,ORD,886,1,FLIES_TO
4,SJC,DEN,948,1,FLIES_TO
5,DEN,SJC,948,1,FLIES_TO
6,SJC,ORD,1829,1,FLIES_TO


In [9]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()

print("--")


/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py:238: UserWarning: unsupported dtype 'object' for 'airport_code' was coerced to 'string'
  warnings.warn(f"unsupported dtype '{original_dtype}' for '{id_col.name}' was coerced to '{id_col.dtype}'")
/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py:238: UserWarning: unsupported dtype 'object' for 'START_ID' was coerced to 'string'
  warnings.warn(f"unsupported dtype '{original_dtype}' for '{id_col.name}' was coerced to '{id_col.dtype}'")
/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/aio/import_data/dataframe_importer/__init__.py:238: UserWarning: unsupported dtype 'object' for 'END_ID' was coerced to 'string'
  warnings.warn(f"unsupported dtype '{original_dtype}' for '{id_col.name}' was coerced to '{id_col.dtype}'")


          0/? [?op/s]

--


In [10]:

#  What's in the graph ..


l_result1 = my_graph.query_unpaginated("""
   MATCH ( n ) 
   RETURN n
   """)

#  print(type(l_result1))
#
#     <class 'katana.remote.ResultSet'>

print(tabulate(l_result1, headers='keys', tablefmt='psql'))

#  Still works
#
#  for l_each in l_result1.itertuples():
#     print(l_each)


l_result2 = my_graph.query_unpaginated("""
   MATCH ( n ) - [ r ] -> ( m )
   RETURN r
   """)
# print(l_result2[0:30])
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


#  Result from nodes ..
#
#  +----+-----------------+-------------+-----------+------------------+--------+-----------------+----------+
#  |    |   n.internal_id | n.labels    | n.LABEL   | n.airport_name   | n.id   |   n.internal_id | n.type   |
#  |----+-----------------+-------------+-----------+------------------+--------+-----------------+----------|
#  |  0 |               0 | ['Airport'] | Airport   | Milwaukee        | MKE    |             600 | node     |
#  |  1 | 281474976710656 | ['Airport'] | Airport   | Chicago O-Hare   | ORD    |             601 | node     |
#  |  2 | 281474976710657 | ['Airport'] | Airport   | San Jose         | SJC    |             602 | node     |
#  |  3 | 562949953421312 | ['Airport'] | Airport   | Denver           | DEN    |             603 | node     |
#  +----+-----------------+-------------+-----------+------------------+--------+-----------------+----------+


#  Result from edges ..
#
#  +----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------+
#  |    |        r.end.id | r.end.labels   |   r.internal_id | r.label   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   |
#  |----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------|
#  |  0 | 281474976710656 | ['Airport']    |               0 | FLIES_TO  |           66 |            1 | FLIES_TO |               0 | ['Airport']      | edge     |
#  |  1 | 562949953421312 | ['Airport']    | 281474976710657 | FLIES_TO  |          886 |            1 | FLIES_TO | 281474976710656 | ['Airport']      | edge     |
#  |  2 |               0 | ['Airport']    | 281474976710659 | FLIES_TO  |           66 |            1 | FLIES_TO | 281474976710656 | ['Airport']      | edge     |
#  |  3 | 281474976710656 | ['Airport']    | 281474976710656 | FLIES_TO  |         1829 |            1 | FLIES_TO | 281474976710657 | ['Airport']      | edge     |
#  |  4 | 562949953421312 | ['Airport']    | 281474976710658 | FLIES_TO  |          948 |            1 | FLIES_TO | 281474976710657 | ['Airport']      | edge     |
#  |  5 | 281474976710657 | ['Airport']    | 562949953421312 | FLIES_TO  |          948 |            1 | FLIES_TO | 562949953421312 | ['Airport']      | edge     |
#  |  6 | 281474976710656 | ['Airport']    | 562949953421313 | FLIES_TO  |          886 |            1 | FLIES_TO | 562949953421312 | ['Airport']      | edge     |
#  +----+-----------------+----------------+-----------------+-----------+--------------+--------------+----------+-----------------+------------------+----------+



          0/? [?op/s]

+----+-----------------+-------------+-----------+------------------+------------------+-----------------+---------------+
|    |           id(n) | labels(n)   | n.LABEL   | n.airport_code   | n.airport_name   |   n.internal_id | datatype(n)   |
|----+-----------------+-------------+-----------+------------------+------------------+-----------------+---------------|
|  0 |               0 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   |             601 | node          |
|  1 | 281474976710656 | ['Airport'] | Airport   | DEN              | Denver           |             603 | node          |
|  2 | 562949953421313 | ['Airport'] | Airport   | SJC              | San Jose         |             602 | node          |
|  3 | 562949953421312 | ['Airport'] | Airport   | MKE              | Milwaukee        |             600 | node          |
+----+-----------------+-------------+-----------+------------------+------------------+-----------------+---------------+


/opt/conda/lib/python3.8/site-packages/katana_enterprise/remote/result_set.py:69: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._response = response


          0/? [?op/s]

+----+------------------+----------------------+-----------------+-----------+--------------+--------------+----------+--------------------+------------------------+---------------+
|    |   id(endNode(r)) | labels(endNode(r))   |           id(r) | type(r)   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |   id(startNode(r)) | labels(startNode(r))   | datatype(r)   |
|----+------------------+----------------------+-----------------+-----------+--------------+--------------+----------+--------------------+------------------------+---------------|
|  0 |  281474976710656 | ['Airport']          |               0 | FLIES_TO  |          886 |            1 | FLIES_TO |                  0 | ['Airport']            | edge          |
|  1 |  562949953421312 | ['Airport']          |               1 | FLIES_TO  |           66 |            1 | FLIES_TO |                  0 | ['Airport']            | edge          |
|  2 |  562949953421313 | ['Airport']          | 281474976710656 | FLIES_TO  |          94